# Layers 部分

## Layer norm

In [ ]:
import torch # 导入 PyTorch 库
from torch import nn # 从 PyTorch 库导入神经网络模块

# LayerNorm 层在神经网络中通常用于稳定和加速训练过程。
# 它通过对每个样本的输入特征进行归一化，使得特征具有零均值和单位方差，
# 从而减少了内部协变量偏移，并且使得不同层之间的特征分布更加一致。这有助于提高模型的泛化能力和收敛速度

class LayerNorm(nn.Module): # 定义一个名为 LayerNorm 的类，继承自 nn.Module
    def __init__(self, d_model, eps=1e-12): # LayerNorm 类的构造函数
        super(LayerNorm, self).__init__() # 初始化父类 nn.Module
        self.gamma = nn.Parameter(torch.ones(d_model)) # 定义可学习参数 gamma，初始化为全 1
        self.beta = nn.Parameter(torch.zeros(d_model)) # 定义可学习参数 beta，初始化为全 0
        self.eps = eps # 定义一个小的常数 eps，防止分母为 0

    def forward(self, x): # LayerNorm 类的前向传播函数
        mean = x.mean(-1, keepdim=True) # 计算输入 x 在最后一个维度上的均值，并保持维度
        var = x.var(-1, unbiased=False, keepdim=True) # 计算输入 x 在最后一个维度上的方差，并保持维度
        # '-1' 表示最后一个维度。

        out = (x - mean) / torch.sqrt(var + self.eps) # 对输入 x 进行层归一化
        out = self.gamma * out + self.beta # 对层归一化后的结果进行缩放和平移
        return out # 返回层归一化后的结果

## Scale dot product attention

In [ ]:
import math  # 导入数学模块以使用平方根函数

from torch import nn  # 从PyTorch库导入神经网络模块


class ScaleDotProductAttention(nn.Module):
    """
    计算缩放点积注意力

    查询：我们关注的给定句子（解码器）
    键：每个句子都要检查与查询的关系（编码器）
    值：每个句子都与键相同（编码器）
    """

    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()  # 初始化父类nn.Module
        self.softmax = nn.Softmax(dim=-1)  # 初始化softmax函数，在最后一个维度上操作

    def forward(self, q, k, v, mask=None, e=1e-12):
        # 输入是4维张量
        # [batch_size, head, length, d_tensor]
        batch_size, head, length, d_tensor = k.size()  # 获取键张量的维度

        # 1. 点积查询与键^T以计算相似度
        k_t = k.transpose(2, 3)  # 转置键张量
        score = (q @ k_t) / math.sqrt(d_tensor)  # 计算缩放点积

        # 2. 应用掩码（可选）
        if mask is not None:
            score = score.masked_fill(mask == 0, -10000)  # 将掩码为0的位置填充为-10000

        # 3. 将它们传递给softmax以使其范围在[0, 1]之间
        score = self.softmax(score)  # 应用softmax函数

        # 4. 乘以值
        v = score @ v  # 计算注意力加权值

        return v, score  # 返回注意力加权值和注意力分数

        # 4. multiply with Value
        v = score @ v

        return v, score

## Multi head attention

In [ ]:
from torch import nn

# from models.layers.scale_dot_product_attention import ScaleDotProductAttention

"""多头注意力机制的核心思想是并行计算多个注意力表示，从而捕捉到输入序列中不同方面的依赖关系。

具体来说，多头注意力机制将输入序列分别通过多个独立的注意力头进行处理。
每个注意力头都有一组独立的参数，可以学习到不同的注意力表示。这些注意力表示最后被拼接起来，并通过一个线性变换得到最终的输出。

这样做的好处主要有以下几点：

捕捉更丰富的上下文信息：不同的注意力头可以关注输入序列的不同部分，从而捕捉到更全面的上下文信息。
提高模型的表达能力：通过并行计算多个注意力表示，模型可以学习到更复杂的特征表示。
增强模型的鲁棒性：多个注意力头可以相互补充，减少单个注意力头出错带来的影响。
"""

class MultiHeadAttention(nn.Module):
    # 多头注意力机制的实现

    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head  # 注意力头的数量
        self.attention = ScaleDotProductAttention()  # 缩放点积注意力层
        self.w_q = nn.Linear(d_model, d_model)  # 查询向量的线性变换
        self.w_k = nn.Linear(d_model, d_model)  # 键向量的线性变换
        self.w_v = nn.Linear(d_model, d_model)  # 值向量的线性变换
        self.w_concat = nn.Linear(d_model, d_model)  # 拼接后的向量的线性变换

    def forward(self, q, k, v, mask=None):
        # 前向传播
        # 1. 使用权重矩阵进行点积
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)  # 对查询、键、值向量进行线性变换

        # 2. 按注意力头的数量分割张量
        q, k, v = self.split(q), self.split(k), self.split(v)  # 将张量分割成多个头

        # 3. 进行缩放点积以计算相似度
        out, attention = self.attention(q, k, v, mask=mask)  # 计算注意力权重和输出

        # 4. 拼接并传递给线性层
        out = self.concat(out)  # 将多个头的输出拼接起来
        out = self.w_concat(out)  # 对拼接后的向量进行线性变换

        # 5. 可视化注意力图
        # TODO : 我们应该实现可视化功能

        return out  # 返回多头注意力机制的输出

    def split(self, tensor):
        # 按注意力头的数量分割张量
        """
        按注意力头的数量分割张量

        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()  # 获取张量的维度

        d_tensor = d_model // self.n_head  # 计算每个头的维度
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        # 类似于分组卷积（按注意力头的数量分割）

        return tensor  # 返回分割后的张量

    def concat(self, tensor):
        # self.split(tensor : torch.Tensor) 的逆函数
        """
        self.split(tensor : torch.Tensor) 的逆函数

        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()  # 获取张量的维度
        d_model = head * d_tensor  # 计算原始张量的维度

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        # 将多个头的输出拼接起来
        return tensor  # 返回拼接后的张量

## Position wise feed forward

In [ ]:
from torch import nn  # 导入 PyTorch 的神经网络模块


class PositionwiseFeedForward(nn.Module):  # 定义一个名为 PositionwiseFeedForward 的类，继承自 nn.Module

    def __init__(self, d_model, hidden, drop_prob=0.1):  # 初始化函数，定义模型的层
        super(PositionwiseFeedForward, self).__init__()  # 初始化父类 nn.Module
        self.linear1 = nn.Linear(d_model, hidden)  # 第一个线性层，将输入维度 d_model 映射到 hidden 维度
        self.linear2 = nn.Linear(hidden, d_model)  # 第二个线性层，将 hidden 维度映射回 d_model 维度
        self.relu = nn.ReLU()  # ReLU 激活函数
        self.dropout = nn.Dropout(p=drop_prob)  # Dropout 层，防止过拟合

# Dropout层是一种正则化技术，用于减少神经网络中的过拟合
# 在训练过程中，Dropout层会随机“丢弃”一部分神经元（及其连接），从而防止神经元之间过度依赖，迫使网络学习更鲁棒的特征表示

    def forward(self, x):  # 定义前向传播函数
        x = self.linear1(x)  # 通过第一个线性层
        x = self.relu(x)  # 应用 ReLU 激活函数
        x = self.dropout(x)  # 应用 Dropout
        x = self.linear2(x)  # 通过第二个线性层
        return x  # 返回输出

# Embedding 部分

## Positional Encoding

In [ ]:
import torch
from torch import nn


class PositionalEncoding(nn.Module):
    """
    计算正弦编码。
    """

    def __init__(self, d_model, max_len, device):
        """
        正弦编码类的构造函数

        :param d_model: 模型的维度
        :param max_len: 最大序列长度
        :param device: 硬件设备设置
        """
        super(PositionalEncoding, self).__init__()

        # 与输入矩阵大小相同（用于与输入矩阵相加）
        self.encoding = torch.zeros(max_len, d_model, device=device)
        self.encoding.requires_grad = False  # 我们不需要计算梯度

        pos = torch.arange(0, max_len, device=device)
        pos = pos.float().unsqueeze(dim=1)
        # 1D => 2D unsqueeze 表示单词的位置

        _2i = torch.arange(0, d_model, step=2, device=device).float()
        # 'i' 表示 d_model 的索引（例如，嵌入大小 = 50，'i' = [0,50]）
        # "step=2" 表示 'i' 乘以 2（与 2 * i 相同）

        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))
        # 计算位置编码以考虑单词的位置信息

    def forward(self, x):
        # self.encoding
        # [max_len = 512, d_model = 512]

        batch_size, seq_len = x.size()
        # [batch_size = 128, seq_len = 30]

        return self.encoding[:seq_len, :]
        # [seq_len = 30, d_model = 512]
        # 它将与 tok_emb 相加：[128, 30, 512]

## Token Embeddings

In [ ]:
from torch import nn


class TokenEmbedding(nn.Embedding):
    """
    使用 torch.nn 进行词元嵌入
    它们将使用加权矩阵对单词进行密集表示
    """

    def __init__(self, vocab_size, d_model):
        """
        包含位置信息的词元嵌入类

        :param vocab_size: 词汇表大小
        :param d_model: 模型维度
        """
        super(TokenEmbedding, self).__init__(vocab_size, d_model, padding_idx=1)

## Transformer Embedding

In [ ]:
from torch import nn # 从 PyTorch 导入神经网络模块。

# from models.embedding.positional_encoding import PositionalEncoding
# from models.embedding.token_embeddings import TokenEmbedding
# 从你的模型定义中导入位置编码和标记嵌入模块。

class TransformerEmbedding(nn.Module):
    """
    token embedding + positional encoding (sinusoid)
    positional encoding can give positional information to network
    """
    # 定义一个名为 TransformerEmbedding 的类，它继承自 nn.Module，表示一个 PyTorch 神经网络模块。
    # 它将标记嵌入和位置编码结合起来，为网络提供位置信息。

    def __init__(self, vocab_size, d_model, max_len, drop_prob, device):
        """
        class for word embedding that included positional information

        :param vocab_size: size of vocabulary
        :param d_model: dimensions of model
        """
        # TransformerEmbedding 类的构造函数，它初始化类的属性。
        # 它接受词汇量大小、模型维度、最大序列长度、dropout 概率和设备作为输入。

        super(TransformerEmbedding, self).__init__()
        # 初始化父类 nn.Module，确保正确设置所有必要的属性和方法。

        self.tok_emb = TokenEmbedding(vocab_size, d_model)
        # 创建一个 TokenEmbedding 层，将标记转换为密集向量表示。

        self.pos_emb = PositionalEncoding(d_model, max_len, device)
        # 创建一个 PositionalEncoding 层，将位置信息添加到标记嵌入中。

        self.drop_out = nn.Dropout(p=drop_prob)
        # 创建一个 dropout 层，以防止过拟合，通过在训练期间随机丢弃一些神经元。

    def forward(self, x):
        # 定义 TransformerEmbedding 模块的前向传递。
        # 它接受输入张量 x 并通过标记嵌入、位置编码和 dropout 层进行传递。

        tok_emb = self.tok_emb(x)
        # 通过 TokenEmbedding 层传递输入，获得标记嵌入。

        pos_emb = self.pos_emb(x)
        # 通过 PositionalEncoding 层传递输入，获得位置编码。

        return self.drop_out(tok_emb + pos_emb)
        # 将标记嵌入和位置编码相加，应用 dropout，并返回结果。

#Blocks 部分

## Decoder layer

In [ ]:
from torch import nn

# from models.layers.layer_norm import LayerNorm
# from models.layers.multi_head_attention import MultiHeadAttention
# from models.layers.position_wise_feed_forward import PositionwiseFeedForward


class DecoderLayer(nn.Module): # 定义一个解码器层类，继承自nn.Module

    def __init__(self, d_model, ffn_hidden, n_head, drop_prob): # 初始化函数，定义模型参数
        super(DecoderLayer, self).__init__() # 初始化父类
        self.self_attention = MultiHeadAttention(d_model=d_model, n_head=n_head) # 定义自注意力机制
        self.norm1 = LayerNorm(d_model=d_model) # 定义层归一化
        self.dropout1 = nn.Dropout(p=drop_prob) # 定义dropout层

        self.enc_dec_attention = MultiHeadAttention(d_model=d_model, n_head=n_head) # 定义编码器-解码器注意力机制
        self.norm2 = LayerNorm(d_model=d_model) # 定义层归一化
        self.dropout2 = nn.Dropout(p=drop_prob) # 定义dropout层

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob) # 定义前馈神经网络
        self.norm3 = LayerNorm(d_model=d_model) # 定义层归一化
        self.dropout3 = nn.Dropout(p=drop_prob) # 定义dropout层

    def forward(self, dec, enc, trg_mask, src_mask): # 定义前向传播函数
        # 1. compute self attention # 计算自注意力
        _x = dec # 保存输入
        x = self.self_attention(q=dec, k=dec, v=dec, mask=trg_mask) # 计算自注意力

        # 2. add and norm # 残差连接和层归一化
        x = self.dropout1(x) # dropout
        x = self.norm1(x + _x) # 层归一化

        if enc is not None: # 如果有编码器输入
            # 3. compute encoder - decoder attention # 计算编码器-解码器注意力
            _x = x # 保存输入
            x = self.enc_dec_attention(q=x, k=enc, v=enc, mask=src_mask) # 计算编码器-解码器注意力

            # 4. add and norm # 残差连接和层归一化
            x = self.dropout2(x) # dropout
            x = self.norm2(x + _x) # 层归一化

        # 5. positionwise feed forward network # 前馈神经网络
        _x = x # 保存输入
        x = self.ffn(x) # 前馈神经网络

        # 6. add and norm # 残差连接和层归一化
        x = self.dropout3(x) # dropout
        x = self.norm3(x + _x) # 层归一化
        return x # 返回输出

## Encoder layer

In [ ]:
from torch import nn

# from models.layers.layer_norm import LayerNorm
# from models.layers.multi_head_attention import MultiHeadAttention
# from models.layers.position_wise_feed_forward import PositionwiseFeedForward


class EncoderLayer(nn.Module):
    """Transformer 编码器层"""

    def __init__(self, d_model, ffn_hidden, n_head, drop_prob):
        """初始化。

        参数：
            d_model: 模型维度
            ffn_hidden: 前馈网络中隐藏层的维度
            n_head: 多头注意力中的头数
            drop_prob: dropout 概率
        """
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, n_head=n_head) # 多头注意力层
        self.norm1 = LayerNorm(d_model=d_model) # 层归一化层
        self.dropout1 = nn.Dropout(p=drop_prob) # dropout 层

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob) # 前馈网络
        self.norm2 = LayerNorm(d_model=d_model) # 层归一化层
        self.dropout2 = nn.Dropout(p=drop_prob) # dropout 层

    def forward(self, x, src_mask):
        """前向传播。

        参数：
            x: 输入张量
            src_mask: 源掩码

        返回：
            输出张量
        """
        # 1. 计算自注意力
        _x = x
        x = self.attention(q=x, k=x, v=x, mask=src_mask) # 计算自注意力

        # 2. 添加残差连接并进行层归一化
        x = self.dropout1(x) # 应用 dropout
        x = self.norm1(x + _x) # 添加残差连接并应用层归一化

        # 3. 应用位置前馈网络
        _x = x
        x = self.ffn(x) # 应用位置前馈网络

        # 4. 添加残差连接并进行层归一化
        x = self.dropout2(x) # 应用 dropout
        x = self.norm2(x + _x) # 添加残差连接并应用层归一化
        return x

# Model 部分

## Decoder

In [ ]:
import torch
from torch import nn

# from models.blocks.decoder_layer import DecoderLayer
# from models.embedding.transformer_embedding import TransformerEmbedding


class Decoder(nn.Module):
    def __init__(self, dec_voc_size, max_len, d_model, ffn_hidden, n_head, n_layers, drop_prob, device):
        # 初始化解码器模块的父类
        super().__init__()
        # 初始化词嵌入层
        self.emb = TransformerEmbedding(d_model=d_model,
                                        drop_prob=drop_prob,
                                        max_len=max_len,
                                        vocab_size=dec_voc_size,
                                        device=device)
        # 初始化解码器层列表
        self.layers = nn.ModuleList([DecoderLayer(d_model=d_model,
                                                  ffn_hidden=ffn_hidden,
                                                  n_head=n_head,
                                                  drop_prob=drop_prob)
                                     for _ in range(n_layers)])
        # 初始化线性层，将解码器输出映射到目标词汇表大小
        self.linear = nn.Linear(d_model, dec_voc_size)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        # 对目标序列进行词嵌入
        trg = self.emb(trg)
        # 通过每一层解码器层传递嵌入的目标序列
        for layer in self.layers:
            trg = layer(trg, enc_src, trg_mask, src_mask)
        # 将最终解码器层的输出传递给线性层
        output = self.linear(trg)
        # 返回线性层的输出
        return output

## Encoder

In [ ]:
from torch import nn # 从 PyTorch 导入神经网络模块

# from models.blocks.encoder_layer import EncoderLayer # 从本地模块导入 EncoderLayer 类
# from models.embedding.transformer_embedding import TransformerEmbedding # 从本地模块导入 TransformerEmbedding 类


class Encoder(nn.Module): # 定义一个继承自 nn.Module 的 Encoder 类

    def __init__(self, enc_voc_size, max_len, d_model, ffn_hidden, n_head, n_layers, drop_prob, device): # 初始化方法
        super().__init__() # 初始化父类 nn.Module
        self.emb = TransformerEmbedding(d_model=d_model, # 创建 TransformerEmbedding 层的实例
                                        max_len=max_len,
                                        vocab_size=enc_voc_size,
                                        drop_prob=drop_prob,
                                        device=device)

        self.layers = nn.ModuleList([EncoderLayer(d_model=d_model, # 创建 EncoderLayer 层的列表
                                                  ffn_hidden=ffn_hidden,
                                                  n_head=n_head,
                                                  drop_prob=drop_prob)
                                     for _ in range(n_layers)])

    def forward(self, x, src_mask): # 定义前向传递方法
        x = self.emb(x) # 将输入传递给嵌入层

        for layer in self.layers: # 遍历 EncoderLayer 层
            x = layer(x, src_mask) # 将输出从前一层传递到下一层

        return x # 返回编码器的最终输出

## Transformer

In [ ]:
import torch # 导入 PyTorch 库，用于构建神经网络。
from torch import nn # 导入 PyTorch 的神经网络模块。

# from models.model.decoder import Decoder # 从 models.model.decoder 模块导入 Decoder 类。
# from models.model.encoder import Encoder # 从 models.model.encoder 模块导入 Encoder 类。


class Transformer(nn.Module): # 定义一个 Transformer 类，继承自 nn.Module。

    def __init__(self, src_pad_idx, trg_pad_idx, trg_sos_idx, enc_voc_size, dec_voc_size, d_model, n_head, max_len,
                 ffn_hidden, n_layers, drop_prob, device): # Transformer 类的构造函数。
        super().__init__() # 初始化父类 nn.Module。
        self.src_pad_idx = src_pad_idx # 源语言填充索引。
        self.trg_pad_idx = trg_pad_idx # 目标语言填充索引。
        self.trg_sos_idx = trg_sos_idx # 目标语言开始符号索引。
        self.device = device # 设备（CPU 或 GPU）。
        self.encoder = Encoder(d_model=d_model, # 创建 Encoder 对象。
                               n_head=n_head,
                               max_len=max_len,
                               ffn_hidden=ffn_hidden,
                               enc_voc_size=enc_voc_size,
                               drop_prob=drop_prob,
                               n_layers=n_layers,
                               device=device)

        self.decoder = Decoder(d_model=d_model, # 创建 Decoder 对象。
                               n_head=n_head,
                               max_len=max_len,
                               ffn_hidden=ffn_hidden,
                               dec_voc_size=dec_voc_size,
                               drop_prob=drop_prob,
                               n_layers=n_layers,
                               device=device)

    def forward(self, src, trg): # Transformer 类的前向传播方法。
        src_mask = self.make_src_mask(src) # 创建源语言掩码。
        trg_mask = self.make_trg_mask(trg) # 创建目标语言掩码。
        enc_src = self.encoder(src, src_mask) # 对源语言进行编码。
        output = self.decoder(trg, enc_src, trg_mask, src_mask) # 对目标语言进行解码。
        return output # 返回解码结果。

    def make_src_mask(self, src): # 创建源语言掩码的方法。
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2) # 计算掩码。
        return src_mask # 返回掩码。

    def make_trg_mask(self, trg): # 创建目标语言掩码的方法。
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(3) # 计算填充掩码。
        trg_len = trg.shape[1] # 目标语言序列长度。
        trg_sub_mask = torch.tril(torch.ones(trg_len, trg_len)).type(torch.ByteTensor).to(self.device) # 计算子序列掩码。
        trg_mask = trg_pad_mask & trg_sub_mask # 结合填充掩码和子序列掩码。
        return trg_mask # 返回掩码。